# Getting Spotify playlist data

This Notebook has the intention to do some basic analysis from my favorite artists from spotify for my application website

## Initialization

In [1]:
import json
import os
import re
import time
from collections import Counter, defaultdict

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from requests.auth import HTTPBasicAuth

In [2]:
os.makedirs("src/assets/analysis/", exist_ok=True);

In [3]:
# Spotify API parameters
SPOTIFY_AUTH_URL     = 'https://accounts.spotify.com/api/token'
SPOTIFY_API_URL = 'https://api.spotify.com/v1'

SPOTIFY_PLAYLIST_ID = os.getenv('PLAYLIST_ID');

## API Auth
Define the Spotify development project secrets as credentials to request from endpoints. For basic usage, a simple auth method is sufficient. However, to add or remove songs from a playlist, for example, an oauth via link is required - which is not needed in this application. More information can be found in [Spotify's API docs](https://developer.spotify.com/documentation/web-api).

In [4]:
client_id = os.getenv('CLIENT_ID');
client_secret = os.getenv('CLIENT_SECRET');

In [5]:
response = requests.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials'
}, auth=(client_id, client_secret))

data = response.json()
access_token = data['access_token']
headers = {
    'Authorization': f'Bearer {access_token}'
}


## Playlist data

First, all playlist tracks are loaded in batches and formatted by album (as album covers are displayed on the website). The track IDs are stored separately for the latter analysis.

In [6]:
import time
import requests

def fetchPlaylistTracks(url, headers, max_retries=3, cooldown=5):
    formatted_data = []
    tracks = []

    while url:
        retries = 0

        while retries < max_retries:
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                data = response.json()
                break  # Exit the retry loop when the request is successful

            elif response.status_code == 429:  # API rate limit exceeded
                print("Rate limit exceeded. Retrying after cooldown...")
                time.sleep(cooldown)
                retries += 1

            else:  # Handle unexpected status codes
                print(f"Unexpected status code: {response.status_code}. Retrying...")
                retries += 1

        else:
            # Raise an error if the max retries are exhausted
            raise Exception(f"Failed to fetch playlist tracks after {max_retries} attempts for URL: {url}")

        # Process data after a successful response
        for item in data['items']:
            track = item['track']
            album = track['album']

            tracks.append(track['id'])
            # Check if the album already exists in formatted_data
            existing_album = next((item for item in formatted_data if item['id'] == album['id']), None)

            if existing_album:
                # If the album exists, add the track to the album's tracks
                if track['id'] not in existing_album['tracks']:
                    existing_album['tracks'].append(track['id'])
            else:
                # If the album does not exist, create a new entry for the album
                formatted_album = {
                    'cover': album['images'][0]['url'].removeprefix('https://i.scdn.co/image/ab67616d0000b273'),
                    'id': album['id'],
                    'name': album['name'],
                    'artists': [{'name': artist['name'], 'id': artist['id']} for artist in album['artists']],
                    'release': album['release_date'],
                    'tracks': [track['id']]
                }

                # Add the constructed album/track data to your list
                formatted_data.append(formatted_album)

        url = data.get('next')  # URL for the next set of items, if any

    return formatted_data, tracks


In [7]:
albums, tracks = fetchPlaylistTracks(f'{SPOTIFY_API_URL}/playlists/{SPOTIFY_PLAYLIST_ID}/tracks', headers);

# stats
print("Albums:" + str(len(albums)));
print("Songs:" + str(len(tracks)))

Albums:796
Songs:1786


In [8]:
with open("export-spotify-covers.json", "w") as outfile:
    json.dump(albums, outfile)

time.sleep(10)

## Artist occurence & genres

The artist profile data is loaded to get the genre information (what genre an artist belongs to), but also to get more information about the artist, like the name and also the link to their profile picture.

In [9]:
# Function to process albums and update stats for favorite artists
def getArtistStats(albums, stats):
    for album in albums:
        album_artist_ids = [artist['id'] for artist in album['artists']]

        for artist_id in album_artist_ids:

            if artist_id not in stats:
                stats[artist_id] = {
                    'img': '',
                    'tracks': len(album['tracks']),
                    'albums': 1,
                    'name': ''
                }
            else:
                stats[artist_id]['tracks'] += len(album['tracks'])
                stats[artist_id]['albums'] += 1

# fetch artist data with retry mechanism
def fetch_artist_data(artist_id, max_retries=3, cooldown=5):
    retries = 0

    while retries < max_retries:
        response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=headers)

        if response.status_code == 200:
            return response.json()

        elif response.status_code == 429:  # API rate limit exceeded
            print("Rate limit exceeded. Retrying after cooldown...")
            time.sleep(cooldown)  # Wait for cooldown period
            retries += 1

        else:  # For other non-200 responses
            print(f"Unexpected status code: {response.status_code}. Retrying...")
            retries += 1

    # After max retries, raise an error
    raise Exception(f"Failed to fetch artist data after {max_retries} attempts")


In [10]:
artists_stats = dict();

# Process albums data structure here
getArtistStats(albums, artists_stats)

genre_track_count = Counter()  # Using a Counter to easily sum values for the same key


# Assuming artistStats is defined somewhere with artist IDs and their track counts
for artist_id in artists_stats:


    artist_data = fetch_artist_data(artist_id)  # Ensure headers is defined with your auth token

    if not artist_data:
        continue

    genres = artist_data.get('genres', [])
    artistTracksCount = artists_stats[artist_id]['tracks']

    # Incrementing track count for each genre found for the artist
    for genre in genres:
        genre_track_count[genre] += artistTracksCount

    # Optionally update image and name if present
    if 'images' in artist_data and artist_data['images']:
        artists_stats[artist_id]['img'] = artist_data['images'][0]['url'].replace('https://i.scdn.co/image/ab6761610000e5eb', '')
        artists_stats[artist_id]['name'] = artist_data.get("name", '')

Rate limit exceeded. Retrying after cooldown...


Rate limit exceeded. Retrying after cooldown...


In [11]:
# add percentage
artists_stats = {artist_id: {**details, 'occurrence': format(((details['tracks'] / len(tracks)) * 100), ".2f")} for artist_id, details in artists_stats.items()}
favorite_artists = sorted(artists_stats.items(), key=lambda x: x[1]['occurrence'], reverse=True)[:5];


### Generalised genres

Since many genres are very specific and cannot be visualised in a meaningful way, they have to be grouped, e.g. German rap, Swiss rap are all in the category rap. Some genres like " antidetusch " cannot be grouped and are put into the category "other". The counts of this category are stored separately to be able to list the "other" genres.

In [12]:
# list of general genres with some additional groupings
general_genres = [
    'pop', 'indie', 'hip hop', 'rock', 'edm', 'jazz', 'reggae', 'singer-songwriter',
    'ambient', 'experimental', 'electronic', 'folk', 'alternative', 'punk', 'ska', 'metal',
    'classical', 'country', 'soul', 'blues', 'r&b', 'comedy', 'grunge', 'new wave'
]

general_genre_counts = Counter()
unmatched_genre_counts = Counter()

def match_general_genre(specific_genre, general_genres):

    # Normalize and consolidate similar genres
    specific_genre = specific_genre.lower()
    if 'electro' in specific_genre or 'dance' in specific_genre:
        return 'electronic'
    if 'hip hop' in specific_genre or 'rap' in specific_genre:
        return 'hip hop/rap'
    if 'motown' in specific_genre or 'soul' in specific_genre or 'r&b' in specific_genre:
        return 'soul/r&b'
    if 'neue deutsche welle' in specific_genre or 'german' in specific_genre:
        return 'alternative'
    if 'grunge' in specific_genre or 'post-grunge' in specific_genre:
        return 'grunge'
    if 'new wave' in specific_genre or 'new romantic' in specific_genre:
        return 'new wave'
    if 'comedy' in specific_genre or 'parody' in specific_genre or 'comic' in specific_genre:
        return 'comedy'

    # check for matches in general genres
    for general_genre in general_genres:
        if general_genre in specific_genre:
            return general_genre

    # track unmatched genres
    unmatched_genre_counts[specific_genre] += 1
    return 'other'

# categorize each specific genre
for specific_genre, count in genre_track_count.items():
    general_genre = match_general_genre(specific_genre, general_genres)
    general_genre_counts[general_genre] += count
    if general_genre == 'other':
        unmatched_genre_counts[specific_genre] += count

general_genre_counts = sorted(general_genre_counts.items(), key=lambda x: x[1], reverse=True)

## Track's audio features

The Spotify API can be used to retrieve a track's features. This data includes danceability, valence, speechiness and so on. This information for each song is retrieved in batches and then averaged to give a score for the whole dataset.

**NOTE:** this feature was surprisingly deprecated and may therefore not used anymore TBD

In [13]:

# deduplicate the track IDs in case the same track appears more than once
unique_track_ids = tracks

#audio_features_data = {}

# spotify limits the number of track IDs you can request at once to 100
#batch_size = 100
#for i in range(0, len(unique_track_ids), batch_size):

#    batch = unique_track_ids[i:i+batch_size]

#    response = requests.get((f'{SPOTIFY_API_URL}/audio-features/'), headers=headers, params={'ids': ','.join(batch)})
#    data = response.json()

    # For each track, fetch its audio features
#    for feature in data['audio_features']:
#        if feature:  # Check if the feature is not None
#           track_id = feature['id']

            # Remove 'track_href' and 'analysis_url' from the feature data
#            feature.pop('track_href', None)  # Use pop to remove 'track_href', if it exists
#            feature.pop('analysis_url', None)  # Use pop to remove 'analysis_url', if it exists
#           feature.pop('uri', None)  # Use pop to remove 'analysis_url', if it exists

#           audio_features_data[track_id] = feature

# Initialize a dictionary to hold the sum of each feature
#features_sum = {key: 0 for key in audio_features_data[list(audio_features_data.keys())[0]].keys() if key != 'type' and key != 'id' and key != 'uri' and key != 'track_href' and key != 'analysis_url' and key != 'duration_ms' and key != 'time_signature'}

# Sum up all the features
#for features in audio_features_data.values():
#    for key, value in features.items():
#        if key in features_sum:
#            features_sum[key] += value

# Calculate the average
#features_avg = {key: (value / len(audio_features_data)) for key, value in features_sum.items()}

## Playlist duration

In [14]:
# fetch data for multiple tracks at once
def fetch_tracks_data(track_ids):
    track_ids_str = ','.join(track_ids)
    response = requests.get(f'https://api.spotify.com/v1/tracks?ids={track_ids_str}', headers=headers)
    if response.status_code == 200:
        return response.json()['tracks']
    else:
        return []

playlist_duration = 0

# Create batches of up to 50 track IDs
batch_size = 50
for i in range(0, len(tracks), batch_size):
    batch = unique_track_ids[i:i+batch_size]
    track_data = fetch_tracks_data(batch)
    for track in track_data:
        if track:  # Ensure the track data is not None
            playlist_duration += track.get('duration_ms', 0)




## Release date grouping


In [15]:
# Initialize a defaultdict to count albums per decade
decade_counts = defaultdict(int)

for details in albums:
    # Extract year from the release_date
    date = pd.to_datetime(details['release'])

    # Convert year to its corresponding decade
    year = date.year
    decade = (year // 10) * 10

    # Count albums per decade
    decade_counts[decade] += len(details['tracks'])

sorted_counts = sorted(decade_counts.items())

# Export resulting analysis data

In [16]:

# reusing the audio-features from the prev analysis as a workaround for now (songs are barely updated anyway)

features_avg = {}
with open("export-spotify-analysis.json") as f:
    prevData = json.load(f)
    features_avg = prevData["features"]




In [17]:
# Constructing the data structure
dataResult = {
    "update_time": int(time.time()),
    "stats": {
        "albums": len(albums),
        "tracks": len(tracks),
        "artists": len(artists_stats),
        "duration": playlist_duration/60000
    },

    "genres": {
        "general": dict(general_genre_counts), #dict(general_genre_counts),
        "detailed": dict(genre_track_count.most_common(10))
    },

    "features": dict(features_avg),
    "artists": dict(favorite_artists),
    "decades": dict(sorted_counts)

}

In [18]:
with open("export-spotify-analysis.json", "w") as outfile:
    json.dump(dataResult, outfile)